In [36]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import sys
!{sys.executable} -m pip install pandas-profiling
!{sys.executable} -m pip install pandasql 
from pandasql import sqldf
import pandas_profiling

###############################################################################################
# Process Clinical data
###############################################################################################

patient_data = pd.read_csv('patient_data.csv')
patient = pd.read_csv('patient_data.csv')

patient = patient.set_index('track_name').T
patient = patient[1:]
patient = patient.reset_index()
patient = patient.rename(columns = {'index':'ID'})

patient['ID'] = patient['ID'].str.upper()
patient['Diagnosis Age'] = pd.to_numeric(patient['Diagnosis Age'])
patient['Overall Survival (Months)'] = pd.to_numeric(patient['Overall Survival (Months)'])
patient['Mutation Count'] = pd.to_numeric(patient['Mutation Count'])
patient['Sex'] = patient['Sex'].str.lower()
patient = patient[patient['# Samples per Patient']!='2'].reset_index(drop=True)

def label(x):
    if 'T1' in x:
        return 'T1'
    if 'T2' in x:
        return 'T2'
    if 'T3' in x:
        return 'T3'
    return 'T4'
patient['Stage'] = patient['American Joint Committee on Cancer Tumor Stage Code'].apply(lambda x: label(x))

patient = patient.drop(['Profiled in Mutations','Profiled in Putative copy-number alterations from GISTIC','Patient\'s Vital Status','# Samples per Patient','Cancer Type','Informed consent verified','Neoplasm Histologic Type Name'],axis=1)

###############################################################################################
# Process Tumor mRNA gene expression data
###############################################################################################

mrna_data = pd.read_csv('mrna_data.txt',sep='\t')
mrna = pd.read_csv('mrna_data.txt',sep='\t')

mrna.dropna(thresh=1,axis=1,inplace=True)
mrna = mrna.drop(['STUDY_ID'],axis=1)

def id_process(x):
    new = x[:len(x)-3]
    new = new.replace("_","")
    return new.upper()
mrna['SAMPLE_ID'] = mrna['SAMPLE_ID'].apply(lambda x: id_process(x))

###############################################################################################
# Process Tumor Mutation Sequencing data
###############################################################################################

seq_data = pd.read_csv('seq_data.txt',sep='\t')
seq = pd.read_csv('seq_data.txt',sep='\t')

seq.dropna(how='all',axis=1,inplace=True)
seq = seq.drop(['STUDY_ID'],axis=1)
seq['SAMPLE_ID'] = seq['SAMPLE_ID'].apply(lambda x: id_process(x))
seq = pd.get_dummies(seq, columns=seq.iloc[:,1:].columns,dummy_na=False)


In [37]:
###############################################################################################
# Process combined dataframe
###############################################################################################

def pysqldf(q):
    return sqldf(q, globals())

q = '''
SELECT *
FROM patient
JOIN mrna
ON patient.ID = mrna.SAMPLE_ID
JOIN seq
ON patient.ID = seq.SAMPLE_ID
'''
df = pysqldf(q)



In [38]:
df

ID American Joint Committee on Cancer Tumor Stage Code  \
0    CG-A3-3387                                                T1a    
1    CG-A3-3374                                                T1b    
2    CG-A3-3363                                                 T2    
3    CG-CZ-5460                                                T3b    
4    CG-BP-4176                                                T1b    
5    CG-CJ-6032                                                 T2    
6    CG-BP-5192                                                T1a    
7    CG-CW-5585                                                T3b    
8    CG-B0-5691                                                T1a    
9    CG-AK-3453                                                 T2    
10   CG-BP-5006                                                T1a    
11   CG-B0-5706                                                 T2    
12   CG-AK-3433                                                 T2    
13   CG-CW-5591                                                T3a    
14   CG-AK-3440                                                T1a    
15   CG-B0-5083                                                T1a    
16   CG-A3-A8OX                                                T1a    
17   CG-CZ-5982                                                T1a    
18   CG-DV-5566                                                T1a    
19   CG-B8-A54D                                                T3a    
20   CG-BP-5196                                                T1a    
21   CG-AK-3428                                                T3b    
22   CG-B0-5098                                                 T1    
23   CG-B4-5834                                                 T1    
24   CG-BP-4351                                                T3a    
25   CG-BP-4775                                                T1a    
26   CG-CJ-5681                                                T3a    
27   CG-B0-4845                                                T3a    
28   CG-B0-5696                                                T3a    
29   CG-CZ-4858                                                 T2    
..          ...                                                ...    
502  CG-B0-4714                                                T3b    
503  CG-B0-4816                                                 T2    
504  CG-B0-5713                                                T3b    
505  CG-B2-4102                                                T1b    
506  CG-B4-5836                                                T1b    
507  CG-B4-5838                                                 T3    
508  CG-B4-5843                                                 T1    
509  CG-B8-4151                                                T3a    
510  CG-B8-5549                                                T1b    
511  CG-BP-4331                                                T1a    
512  CG-BP-4337                                                T3b    
513  CG-BP-4347                                                T3b    
514  CG-BP-4758                                                T1a    
515  CG-BP-4762                                                T1a    
516  CG-BP-4781                                                T1a    
517  CG-BP-4787                                                T3a    
518  CG-BP-4790                                                T1a    
519  CG-BP-4964                                                T1a    
520  CG-BP-4998                                                T1a    
521  CG-BP-5195                                                T1a    
522  CG-CJ-4902                                                T3a    
523  CG-CJ-4905                                                T1a    
524  CG-CJ-4908                                                T1a    
525  CG-CJ-5686                                                T1b    
526  CG-CW-5589                      

In [39]:
df = df.drop(['SAMPLE_ID'],axis=1)

In [40]:
df.to_csv ('df.csv')

In [41]:
col_with_null = df.loc[:,df.isnull().sum(axis=0)>0]
nulls = col_with_null[col_with_null.isnull().any(axis=1)]
ind = nulls.index

df.iloc[ind,4:6]

Neoplasm Histologic Grade              Race Category
30                         G2                       None
64                       None  BLACK OR AFRICAN AMERICAN
88                         G2                       None
158                        G3                       None
179                        G2                       None
196                      None  BLACK OR AFRICAN AMERICAN
275                      None                      WHITE
276                        G2                       None
349                        G3                       None
483                        G2                       None

In [42]:
from sklearn.neighbors import KNeighborsClassifier

In [43]:
fitting = df
fitting = fitting.iloc[:,:90]
fitting = fitting.drop(['ID','American Joint Committee on Cancer Tumor Stage Code','Overall Survival Status','Sex','Stage'],axis=1)

In [54]:
race_fit = fitting.drop(['Neoplasm Histologic Grade'],axis=1)
null_index = race_fit.isnull().any(axis=1)
row_with_null = race_fit[null_index]
row_no_null = race_fit[-null_index]

clf = KNeighborsClassifier(3, weights='distance')
x = row_no_null.drop(['Race Category'],axis=1)
y = row_no_null['Race Category']
clf.fit(x,y)
x_test = row_with_null.drop(['Race Category'],axis=1)
imputes = clf.predict(x_test)

df.loc[null_index,'Race Category'] = imputes 

/Users/yuchen/anaconda3/envs/ds_pack/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [55]:
grade_fit = fitting.drop(['Race Category'],axis=1)
null_index = grade_fit.isnull().any(axis=1)
row_with_null = grade_fit[null_index]
row_no_null = grade_fit[-null_index]

clf = KNeighborsClassifier(3, weights='distance')
x = row_no_null.drop(['Neoplasm Histologic Grade'],axis=1)
y = row_no_null['Neoplasm Histologic Grade']
clf.fit(x,y)
x_test = row_with_null.drop(['Neoplasm Histologic Grade'],axis=1)
imputes = clf.predict(x_test)

df.loc[null_index,'Neoplasm Histologic Grade'] = imputes 

/Users/yuchen/anaconda3/envs/ds_pack/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [56]:
df.iloc[ind,4:6]

Neoplasm Histologic Grade              Race Category
30                         G2                      WHITE
64                         G1  BLACK OR AFRICAN AMERICAN
88                         G2                      WHITE
158                        G3                      WHITE
179                        G2                      WHITE
196                        G2  BLACK OR AFRICAN AMERICAN
275                        G3                      WHITE
276                        G2                      WHITE
349                        G3                      WHITE
483                        G2                      WHITE

In [57]:
fitting.iloc[ind,2:4]

Neoplasm Histologic Grade              Race Category
30                         G2                       None
64                       None  BLACK OR AFRICAN AMERICAN
88                         G2                       None
158                        G3                       None
179                        G2                       None
196                      None  BLACK OR AFRICAN AMERICAN
275                      None                      WHITE
276                        G2                       None
349                        G3                       None
483                        G2                       None

In [58]:
fitting.isnull().sum().sum(axis=0)

10

In [59]:
df.isnull().sum().sum(axis=0)

0

In [92]:
df_temp = df
df_temp = df.drop(['ID','American Joint Committee on Cancer Tumor Stage Code'],axis=1)

In [93]:
x = df_temp.drop(['Overall Survival (Months)','Overall Survival Status'],axis=1)
x = pd.get_dummies(x, columns=['Sex','Race Category','Neoplasm Histologic Grade','Stage'])

In [94]:
y = df_temp[['Overall Survival (Months)','Overall Survival Status']]
y.loc[:,['Overall Survival Status']] = np.where(y['Overall Survival Status'] == 'LIVING',1,0)

/Users/yuchen/anaconda3/envs/ds_pack/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [95]:
survival_rate = y['Overall Survival Status'].sum()/y['Overall Survival Status'].shape[0]
survival_rate

0.6710526315789473

In [96]:
numeric_features = x.iloc[:,:82]
corr = numeric_features.corr()
sns.heatmap(corr)

In [97]:
y
# why there are some points with 0 Overall Survival (Months) but status is LIVING?

Overall Survival (Months)  Overall Survival Status
0                        20.27                        1
1                        43.17                        1
2                        10.48                        1
3                        94.38                        1
4                        64.22                        1
5                       119.55                        1
6                        23.46                        1
7                        85.71                        1
8                       112.71                        1
9                        83.15                        1
10                       27.60                        1
11                      105.29                        1
12                      111.99                        1
13                       74.61                        1
14                       94.12                        1
15                       34.33                        0
16                        0.00                        1
17                       80.12                        1
18                       45.93                        1
19                       27.27                        1
20                       33.44                        1
21                      122.47                        1
22                       52.04                        0
23                        1.25                        1
24                       31.87                        1
25                       60.55                        1
26                       18.13                        0
27                       65.24                        0
28                       85.71                        1
29                       69.15                        0
..                         ...                      ...
502                       3.25                        0
503                      45.04                        0
504                      91.39                        1
505                      31.27                        1
506                       4.63                        1
507                       5.45                        1
508                       0.36                        1
509                      42.67                        1
510                       6.37                        1
511                      80.62                        0
512                       0.07                        0
513                      44.91                        1
514                      72.54                        1
515                      44.12                        0
516                      68.33                        1
517                      15.77                        0
518                      36.50                        0
519                      61.17                        1
520                      30.62                        1
521                      24.61                        1
522                      49.93                        1
523                      49.15                        1
524                      50.30                        1
525                      66.95                        1
526                      78.12                        1
527                      63.34                        1
528                      55.29                        1
529                      43.66                        1
530                      66.23                        1
531                      19.42                        1

[532 rows x 2 columns]

In [98]:
from sklearn import model_selection

# Reserve 20% for testing
x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y, test_size=0.2)

print('training data has %d observation with %d features'% x_train.shape)
print('test data has %d observation with %d features'% x_test.shape)

training data has 425 observation with 201 features
test data has 107 observation with 201 features


Diagnosis Age  Mutation Count    CASR    BEX1    MYH8   SPON1   FABP4  \
475             68              39 -0.1422 -0.3978 -0.3230 -0.5166 -0.3193   
368             62              38 -0.3270 -0.3801 -0.3371  0.0926 -0.2023   
170             41               0 -0.3270 -0.2782 -0.3319 -0.2078 -0.4125   
267             72              17  0.1434 -0.3884  0.7342 -0.4666 -0.3134   
80              61              72 -0.2866 -0.3843 -0.3325  2.7338  0.0243   
2               50            1392 -0.2440 -0.3324  0.4572  9.8551 -0.4222   
196             61               0 -0.3229  0.0485 -0.3324 -0.6942 -0.3025   
397             60              34  0.0220 -0.3598 -0.1167 -0.4762 -0.3780   
167             69              26 -0.3150 -0.3971 -0.2756 -0.6867 -0.4007   
357             79              57 -0.3231 -0.3909 -0.3360  0.0804 -0.4122   
442             82               0  0.8895 -0.1888 -0.3245  0.5387  0.7739   
175             61              52 -0.3182 -0.3961 -0.3371 -0.4119 -0.0630   
370             65               0 -0.2323 -0.3949 -0.3371 -0.6400 -0.2106   
184             72              77  0.3960 -0.3785  1.1138 -0.2245 -0.1421   
423             72              35 -0.3048 -0.1618 -0.3129 -0.6357 -0.2960   
487             72              30  0.0815 -0.0080 -0.3371 -0.5571 -0.4017   
316             62              19 -0.0857 -0.3785 -0.3315 -0.5957 -0.4075   
506             61              81  0.2401 -0.3954 -0.3361  1.7731  0.1810   
362             46              41 -0.2619 -0.3668 -0.3371  0.5819 -0.4084   
366             67               0 -0.3215 -0.3816 -0.3371 -0.6954  0.4530   
508             45              52 -0.3160 -0.3666  0.0415 -0.6154  0.8934   
385             42              19 -0.0320  1.4774 -0.2343  0.0955 -0.3105   
144             56              41 -0.1068 -0.3507 -0.3358 -0.6697  0.3095   
40              67              56 -0.3237  2.9688 -0.3275  0.1476 -0.4001   
74              64              77 -0.1922 -0.1898 -0.3362  2.2219 -0.4200   
500             72              58 -0.3210  0.2395 -0.3371 -0.6863  0.9282   
136             70              56 -0.3207 -0.3726 -0.3371 -0.6191 -0.0702   
346             65              17  0.9481 -0.2231 -0.3318 -0.5076 -0.3847   
10              61              38  2.5223 -0.2055 -0.3253 -0.1499  0.1199   
201             67              59 -0.2817 -0.3992 -0.3371 -0.6248 -0.4222   
..             ...             ...     ...     ...     ...     ...     ...   
495             49               0 -0.2996 -0.1519 -0.0334 -0.1904 -0.3269   
72              48              38 -0.2926 -0.0522 -0.2544  2.9263 -0.3991   
99              68              60 -0.0956 -0.3099 -0.3371 -0.1113 -0.0936   
41              79              52 -0.1749  2.3125 -0.2262 -0.4732 -0.0568   
287             61              19 -0.2638 -0.3840 -0.3371 -0.6221 -0.3139   
514             40               1 -0.1417 -0.3662  1.5591 -0.0767  0.2544   
463             55              77 -0.1889 -0.3512  0.8027 -0.6664 -0.3727   
409             71              62 -0.3072 -0.1562 -0.3303 -0.2722  0.3496   
116             86               0  0.0112 -0.3787  0.5219  1.7181 -0.3474   
57              73              72 -0.1344 -0.3992  3.3454 -0.3568 -0.1866   
60              65              94 -0.1973 -0.3992  3.3627 -0.6901 -0.4222   
239             71             410  1.1793  0.9994 -0.2976 -0.5612 -0.3238   
279             63              38  0.0462 -0.2465 -0.3346 -0.5456 -0.3559   
388             73              46  0.6195  0.0780 -0.2792  0.8599 -0.1706   
240             63               0  0.0850 -0.3667 -0.3360 -0.5299  0.3448   
426             80             553 -0.3254 -0.3940  0.6381  1.3404  0.0955   
307             62              43 -0.0934 -0.3159 -0.3241 -0.6201 -0.4026   
410             49               0 -0.3178 -0.3160 -0.1738 -0.5796 -0.4206   
82              40              17 -0.2738 -0.3285 -0.3308  2.7495 -0.1413   
454             64         

In [105]:
# Scale the data, using standardization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train.iloc[:,:82] = scaler.fit_transform(x_train.iloc[:,:82])
x_test.iloc[:,:82] = scaler.transform(x_test.iloc[:,:82])

/Users/yuchen/anaconda3/envs/ds_pack/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/yuchen/anaconda3/envs/ds_pack/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Users/yuchen/anaconda3/envs/ds_pack/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/yuchen/anaconda3/envs/ds_pack/lib/python3.6/site-packages/ipykernel_launcher.py:5: DataConversionWarning: Data with input d